In [1]:
"""
@authors: faurand, chardes, ehagensieker
"""
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import math
import datetime
import tqdm
import pprint

# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

In [2]:
# load the dataset 
(train_ds, test_ds) = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

2022-11-27 17:56:10.559164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-27 17:56:10.560612: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
def calc(task,mnist):
    """
    define the two subtasks which differ there target values

    Args: 
    task(int): which task to perform
    mnist(array): dataset to be used for the task
    Return: 
    mnist(array): preprocessed dataset
    """
    if task == 1:
        mnist = mnist.map(lambda pic1,pic2: (pic1[0],pic2[0],tf.cast((pic1[1] + pic2[1] >= 5), tf.int32)))
        return mnist
    if task == 2:
        mnist = mnist.map(lambda pic1,pic2: (pic1[0],pic2[0], tf.cast((pic1[1]-pic2[1]), tf.int32)))
        return mnist

In [4]:
def prepare_mnist_data(task,batch_size,mnist):
  
    #flatten the images into one dimensional vector
    mnist = mnist.map(lambda img, target: (tf.reshape(img, (-1,)), target))
    #convert data from uint8 to float32
    mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
    #normalization to have input of range [-1,1]
    mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
    #create a tuple with shuffled data points 
    mnist = tf.data.Dataset.zip((mnist.shuffle(1000), mnist.shuffle(1000)))
    #create a triple depending on the task to be performed
    mnist = calc(task,mnist)
    #cache this progress in memory - improve performance
    mnist = mnist.cache()
    #shuffle, batch, prefetch
    mnist = mnist.shuffle(1000)
    mnist = mnist.batch(batch_size)
    mnist = mnist.prefetch(tf.data.AUTOTUNE)

    #return preprocessed dataset
    return mnist

In [5]:
"""
@authors: faurand, chardes, ehagensieker
"""
class TwinModel(tf.keras.Model):
    """
    The model we are using for predicting the outcome 
    """ 
    def __init__(self, task=1,optimizer=tf.keras.optimizers.Adam()):
        #inherit from parent class
        super(TwinModel, self).__init__()

        #depending on the task to perform we need different activation functions for the output layer 
        #and different metrics
        if task==1:
            self.metrics_list = [tf.keras.metrics.Mean(name = 'loss'),tf.keras.metrics.BinaryAccuracy(name = "acc")]
            self.loss_function = tf.keras.losses.BinaryCrossentropy()
            self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
            self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
            self.out_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

        if task==2:
            self.metrics_list = [tf.keras.metrics.Mean(name = 'loss'),tf.keras.metrics.MeanAbsoluteError(name = "MAD")]
            self.loss_function = tf.keras.losses.MeanSquaredError()
            self.dense1 = tf.keras.layers.Dense(256, activation= tf.nn.relu)
            self.dense2 = tf.keras.layers.Dense(256, activation= tf.nn.relu)
            self.out_layer = tf.keras.layers.Dense(1, activation= "linear")

        self.optimizer = optimizer

    @tf.function 
    def call(self, images):
        """
        how to forward the images through the layer
        """
        img1, img2 = images 

        img1_x = self.dense1(img1)
        img1_x = self.dense2(img1_x)

        img2_x = self.dense1(img2)
        img2_x = self.dense2(img2_x)

        combined = tf.concat([img1_x, img2_x], axis = 1)
        out = self.out_layer(combined)

        return out

    @property
    def metrics(self):
        return self.metrics_list


    def reset_metric(self):
        for metric in self.metrics: 
            metric.reset_states()

    @tf.function 
    def train_step(self, data):
        img1, img2, t = data

        with tf.GradientTape() as tape:
            output = self((img1, img2), training = True)
            loss = self.loss_function(t, output)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        self.metrics[0].update_state(loss)

        for metric in self.metrics[1:]:
              metric.update_state(t,output)

        return {metric.name: metric.result() for metric in self.metrics}

    @tf.function
    def test_step(self, data):
        img1, img2, t = data

        output = self((img1,img2), training = False)
        loss = self.loss_function(t, output)

        self.metrics[0].update_state(loss)

        for metric in self.metrics[1:]:
              metric.update_state(t,output)

        return {metric.name: metric.result() for metric in self.metrics}

In [6]:
def training_loop(model, train_ds, val_ds, epochs, train_summary_writer, val_summary_writer): 
    #iterate over given amount of epochs
    for epoch in range(epochs): 
        print(f"Epoch {epoch}: ")

        #train on all batches of the training data
        for data in tqdm.tqdm(train_ds, position = 0, leave = True):
            metrics = model.train_step(data)
            
            with train_summary_writer.as_default(): 
                for metric in model.metrics: 
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        # print the metrics
        print([f"Train {key}: {value.numpy()}" for (key, value) in metrics.items()])

        #reset metric 
        model.reset_metrics()

        #evaluation on validation set
        for data in val_ds:
            metrics = model.test_step(data)
            with val_summary_writer.as_default():
                for metric in model.metrics:
                    tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

        # print the metrics
        print([f"Test {key}: {value.numpy()}" for (key, value) in metrics.items()])

        #reset metric
        model.reset_metric()

        print("\n")

In [7]:
def create_summary_writers(config_name):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    train_log_path = f"logs/submission/{config_name}/{current_time}/train"
    val_log_path = f"logs/submission/{config_name}/{current_time}/val"

    # log writer
    train_summary_writer = tf.summary.create_file_writer(train_log_path)
    val_summary_writer = tf.summary.create_file_writer(val_log_path)
    
    return train_summary_writer, val_summary_writer

In [8]:
def run(task, optimizer=tf.keras.optimizers.Adam()):
    ### Hyperparameters
    num_epochs = 15
    batch_size = 32

    train_dataset = prepare_mnist_data(task,batch_size,train_ds)
    test_dataset = prepare_mnist_data(task,batch_size,test_ds)
    
    # Check the dataset
    #print(f'Dataset for subtask {task}: ')
    #for train, test in zip(train_dataset.take(1), test_dataset.take(1)):
    #    print(f'train_img1: {train[0].shape} \ntrain_img2: {train[1].shape} \ntrain_target: {train[2].shape}')
    #    print(f'test_img1: {test[0].shape} \ntest_img2: {test[1].shape} \ntest_target: {test[2].shape}')

    # Initialize the model.
    model = TwinModel(task, optimizer)
    
    # create new summary_writers
    train_summary_writer, val_summary_writer = create_summary_writers(config_name=f'RUN_SUBTASK_{task}')
    
    # Training the model
    training_loop(model,train_dataset,test_dataset,num_epochs,train_summary_writer, val_summary_writer)

In [9]:
print("Task 1, optimizer=SGD, learning_rate=0.001\n")
run(1,tf.keras.optimizers.SGD(learning_rate=0.001))
print("Task 1, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(1,tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9))
print("\nTask 1, optimizer=Adam, learning_rate=0.001\n")
run(1)
print("Task 1, optimizer=RMSprop, learning_rate=0.001\n")
run(1,tf.keras.optimizers.RMSprop(learning_rate=0.001))
print("Task 1, optimizer=Adagrad, learning_rate=0.001\n")
run(1,tf.keras.optimizers.Adagrad(learning_rate=0.001))

Task 1, optimizer=SGD, learning_rate=0.001

Epoch 0: 


100%|███████████████████████████████████████| 1875/1875 [00:32<00:00, 56.88it/s]


['Train loss: 0.3416014313697815', 'Train acc: 0.8556833267211914']
['Test loss: 0.29073646664619446', 'Test acc: 0.8784999847412109']


Epoch 1: 


100%|███████████████████████████████████████| 1875/1875 [00:19<00:00, 94.14it/s]


['Train loss: 0.2684992551803589', 'Train acc: 0.8919166922569275']
['Test loss: 0.2580743730068207', 'Test acc: 0.8967000246047974']


Epoch 2: 


...

['Train loss: 0.15468282997608185', 'Train acc: 0.9413666725158691']
['Test loss: 0.1518036425113678', 'Test acc: 0.9416000247001648']


Epoch 13: 


100%|███████████████████████████████████████| 1875/1875 [00:20<00:00, 91.62it/s]


['Train loss: 0.15138961374759674', 'Train acc: 0.9422833323478699']
['Test loss: 0.14893004298210144', 'Test acc: 0.9427000284194946']


Epoch 14: 


100%|██████████████████████████████████████| 1875/1875 [00:18<00:00, 102.59it/s]


['Train loss: 0.14841867983341217', 'Train acc: 0.9439333081245422']
['Test loss: 0.14704066514968872', 'Test acc: 0.9437000155448914']




In [10]:
print("Task 2, optimizer=SGD, learning_rate=0.001\n")
run(2,tf.keras.optimizers.SGD(learning_rate=0.001))
print("Task 2, optimizer=SGD, learning_rate=0.001, momentum=0.9\n")
run(2,tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9))
print("Task 2, optimizer=Adam, learning_rate=0.001\n")
run(2)
print("Task 2, optimizer=RMSprop, learning_rate=0.001\n")
run(2,tf.keras.optimizers.RMSprop(learning_rate=0.001))
print("Task 2, optimizer=Adagrad, learning_rate=0.001\n")
run(2,tf.keras.optimizers.Adagrad(learning_rate=0.001))

Task 2, optimizer=SGD, learning_rate=0.001

Epoch 0: 


100%|███████████████████████████████████████| 1875/1875 [00:23<00:00, 79.97it/s]


['Train loss: 4.486209392547607', 'Train MAD: 1.6087172031402588']
['Test loss: 2.725328207015991', 'Test MAD: 1.2292664051055908']


Epoch 1: 


100%|██████████████████████████████████████| 1875/1875 [00:15<00:00, 118.84it/s]


['Train loss: 2.3490078449249268', 'Train MAD: 1.1410865783691406']
['Test loss: 2.13855242729187', 'Test MAD: 1.0633130073547363']


Epoch 2: 


...

['Train loss: 1.7716279029846191', 'Train MAD: 0.958962619304657']
['Test loss: 1.9257584810256958', 'Test MAD: 0.9953769445419312']


Epoch 13: 


100%|██████████████████████████████████████| 1875/1875 [00:17<00:00, 106.16it/s]


['Train loss: 1.7324270009994507', 'Train MAD: 0.9468864798545837']
['Test loss: 1.901303768157959', 'Test MAD: 0.9863314628601074']


Epoch 14: 


100%|███████████████████████████████████████| 1875/1875 [00:20<00:00, 91.59it/s]


['Train loss: 1.6970796585083008', 'Train MAD: 0.9360703229904175']
['Test loss: 1.876502275466919', 'Test MAD: 0.9783434271812439']




In [11]:
%tensorboard --logdir logs/submission/